In [6]:
# Import required libraries
import os
import warnings
import sys
import pandas as pd
import numpy as np
from itertools import cycle
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import lasso_path, enet_path
from sklearn import datasets
import datetime
import pickle
from joblib import dump, load
import streamlit as st

# Import mlflow
import mlflow
import mlflow.sklearn
from mlflow import log_metric, log_param, log_artifacts
from mlflow.models.signature import infer_signature
mlflow.set_tracking_uri("http://127.0.0.1:5000/")
mlflow.set_experiment("Red-Blue-Experiment-4")

# Load diabetes dataset
diabetes = datasets.load_diabetes()
X = diabetes.data
y = diabetes.target

# Create pandas DataFrame 

Y = np.array([y]).transpose()
d = np.concatenate((X, Y), axis=1)
cols = ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6', 'progression']
data = pd.DataFrame(d, columns=cols)


In [7]:
diabetes.data[0,:]

array([ 0.03807591,  0.05068012,  0.06169621,  0.02187239, -0.0442235 ,
       -0.03482076, -0.04340085, -0.00259226,  0.01990749, -0.01764613])

comandos mlflow

mlflow server --backend-store-uri sqlite:///mlruns.db --default-artifact-root file://$PWD/mlruns

mlflow models serve --model-uri runs:/ede4cc6ace71468ab2e881dc14d25f6b/models --port 5050


uvicorn app:app --port 5000

docker build -t diabetes-ml-build .
docker run -d -p 80:80 --name diabetes-api iris-ml-build

In [8]:
def plot_enet_descent_path(X, y, l1_ratio):
    # Compute paths
    eps = 5e-3  # the smaller it is the longer is the path
    # Reference the global image variable

    global image
    print("Computing regularization path using ElasticNet.")
    alphas_enet, coefs_enet, _ = enet_path(X, y, eps=eps, l1_ratio=l1_ratio, fit_intercept=False)

    # Display results
    fig = plt.figure(1)
    ax = plt.gca()

    colors = cycle(['b', 'r', 'g', 'c', 'k'])
    neg_log_alphas_enet = -np.log10(alphas_enet)
    for coef_e, c in zip(coefs_enet, colors):
        l1 = plt.plot(neg_log_alphas_enet, coef_e, linestyle='--', c=c)

    plt.xlabel('-Log(alpha)')
    plt.ylabel('coefficients')
    title = 'ElasticNet Path by alpha for l1_ratio = ' + str(l1_ratio)
    plt.title(title)
    plt.axis('tight')

    # Display images
    image = fig

    # Save figure
    fig.savefig("ElasticNet-paths.png")

    # Close plot
    plt.close(fig)

    # Return images
    return image    

In [13]:
# train_diabetes
#   Uses the sklearn Diabetes dataset to predict diabetes progression using ElasticNet
#       The predicted "progression" column is a quantitative measure of disease progression one year after baseline
#       http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html

import joblib


def train_diabetes(data, in_alpha, in_l1_ratio):

  # Evaluate metrics
  def eval_metrics(actual, pred):
      rmse = np.sqrt(mean_squared_error(actual, pred))
      mae = mean_absolute_error(actual, pred)
      r2 = r2_score(actual, pred)
      return rmse, mae, r2

  warnings.filterwarnings("ignore")
  np.random.seed(40)

  # Split the data into training and test sets. (0.75, 0.25) split.
  train, test = train_test_split(data)

  # The predicted column is "progression" which is a quantitative measure of disease progression one year after baseline
  train_x = train.drop(["progression"], axis=1)
  test_x = test.drop(["progression"], axis=1)
  train_y = train[["progression"]]
  test_y = test[["progression"]]

  if float(in_alpha) is None:
    alpha = 0.05
  else:
    alpha = float(in_alpha)

  if float(in_l1_ratio) is None:
    l1_ratio = 0.05
  else:
    l1_ratio = float(in_l1_ratio)


  tag = {"data":"sample", "model": "Elasticnet"}
  runname = "ElasticNet-" + str(datetime.datetime.now()).replace(" ","T")
  with mlflow.start_run(run_name=runname) as run:
    mlflow.set_tags(tag)   
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)
    predicted_qualities = lr.predict(test_x)
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    # Print out ElasticNet model metrics
    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    # Log mlflow attributes for mlflow UI
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    mlflow.sklearn.log_model(lr, "model")

    dump(lr, './ml/diabetes_v2.joblib')
    
  mlflow.end_run()
  print("Training Complete.")

In [14]:
train_diabetes(data,0.01,0.01)
train_diabetes(data,0.02,0.05)
train_diabetes(data,0.2,0.1)

Elasticnet model (alpha=0.010000, l1_ratio=0.010000):
  RMSE: 71.40365347371498
  MAE: 60.09131524886582
  R2: 0.2291124645214172
Training Complete.
Elasticnet model (alpha=0.020000, l1_ratio=0.050000):
  RMSE: 74.8016687900868
  MAE: 63.117724609774626
  R2: 0.15399538913223376
Training Complete.
Elasticnet model (alpha=0.200000, l1_ratio=0.100000):
  RMSE: 81.36629332080149
  MAE: 68.82388164338657
  R2: -0.0010118534327692164
Training Complete.


In [15]:
data.iloc[0]

age              0.038076
sex              0.050680
bmi              0.061696
bp               0.021872
s1              -0.044223
s2              -0.034821
s3              -0.043401
s4              -0.002592
s5               0.019907
s6              -0.017646
progression    151.000000
Name: 0, dtype: float64